# ECE 470 Midterm 2 - Graham Grayson - V00878518

### Question 1
(a) Random Forest is a machine learning technique that works by randomly generating multiple decision trees and then returning the class that is the mode of the classes for classification or the mean of the classes for regression. It introduces a degree of randomness to the tree creation process that helps prevent overfitting. It works based on the idea that a group of weak decision trees can together become a strong decision tree.

(b) Random Forest works best in classification problems, although it can work for regression as well. It works well for large datasets because the training time is fairly short. Random Forests are also able to deal with unbalanced and missing data. It is also easy to implement since it uses a low number of hyperparameters.

(c) Yes, Random Forests can be used to rank feature importance. The approach described in the original paper on Random Forests is to first fit a forest while recording the error of each data point. This error is averaged over the entire forest. Then, the values of each feature being tested are permuted and the model is fitted again, and the error is again recorded and compared against the permuted values. For important features, the permutation of values should have a large impact on accuracy, while the opposite is true for unimportant features.

### Question 2
Note: This is partially based on a project I did for SENG474 (Datamining), which can be found at https://github.com/CallumThomas12/datamining-project-17

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

print(pd.__version__)
#Note: I converted the spreadsheet to an SQLite dababase so I can use SQL
conn = sqlite3.connect("C:\cars.db")
tables = pd.read_sql("Select * from sqlite_master where type = 'table';", conn)

tables

0.25.0


,type,name,tbl_name,rootpage,sql
0,table,craigslistVehiclesFull,craigslistVehiclesFull,2,"CREATE TABLE ""craigslistVehiclesFull"" (\n""url""..."


(a) I wrote an SQL query to extract the Los Angeles data (WHERE city = 'losangeles')

(b) To start with I included the data features that seemed like they could be correlated to price. Features that were omitted were mainly done so because they had too many unique values that made them impossible to classify into numeric data, such as make. For the non-numeric features, I classified them into numeric categories to make it so the classifier can use them.

Later on I tried calculating the correlation to find the best features, this can be found near the bottom of the notebook.

(c) I checked for outliers in the numeric data and added upper/lower bounds on the price and odometer values. I dealt with missing data by specifying that all included features must have non-null values. This shrinks the dataset but also ensures that all the included data is complete. 

In [2]:
query = """
SELECT 
price,
year,
CASE
	WHEN manufacturer = 'acura' THEN 1
	WHEN manufacturer = 'alfa-romeo' THEN 2
    WHEN manufacturer = 'aston' THEN 3
    WHEN manufacturer = 'aston-martin' THEN 3
    WHEN manufacturer = 'audi' THEN 4
    WHEN manufacturer = 'bmw' THEN 5
    WHEN manufacturer = 'buick' THEN 6
    WHEN manufacturer = 'cadillac' THEN 7
    WHEN manufacturer = 'chev' THEN 8
    WHEN manufacturer = 'chevrolet' THEN 8
    WHEN manufacturer = 'chevy' THEN 8
    WHEN manufacturer = 'chrysler' THEN 9
    WHEN manufacturer = 'datsun' THEN 10
    WHEN manufacturer = 'dodge' THEN 11
    WHEN manufacturer = 'ferrari' THEN 12
    WHEN manufacturer = 'fiat' THEN 13
    WHEN manufacturer = 'ford' THEN 14
    WHEN manufacturer = 'gmc' THEN 15
    WHEN manufacturer = 'harley-davidson' THEN 16
    WHEN manufacturer = 'honda' THEN 17
    WHEN manufacturer = 'hyundai' THEN 18
    WHEN manufacturer = 'infiniti' THEN 19
    WHEN manufacturer = 'infinity' THEN 19
    WHEN manufacturer = 'jaguar' THEN 20
    WHEN manufacturer = 'jeep' THEN 21
    WHEN manufacturer = 'kia' THEN 22
    WHEN manufacturer = 'land rover' THEN 23
    WHEN manufacturer = 'rover' THEN 23
    WHEN manufacturer = 'lexus' THEN 24
    WHEN manufacturer = 'lincoln' THEN 25
    WHEN manufacturer = 'mazda' THEN 26
    WHEN manufacturer = 'mercedes' THEN 27
    WHEN manufacturer = 'mercedes-benz' THEN 27
    WHEN manufacturer = 'mercury' THEN 28
    WHEN manufacturer = 'mini' THEN 29
    WHEN manufacturer = 'mitsubishi' THEN 30
    WHEN manufacturer = 'nissan' THEN 31
    WHEN manufacturer = 'pontiac' THEN 32
    WHEN manufacturer = 'porche' THEN 33
    WHEN manufacturer = 'ram' THEN 34
    WHEN manufacturer = 'saturn' THEN 35
    WHEN manufacturer = 'subaru' THEN 36
    WHEN manufacturer = 'toyota' THEN 37
    WHEN manufacturer = 'vw' THEN 38
    WHEN manufacturer = 'volkswagen' THEN 38
    WHEN manufacturer = 'volvo' THEN 39
END AS manufacturer,
CASE
    WHEN condition = 'new' THEN 1
    WHEN condition = 'like new' THEN 2
    WHEN condition = 'excellent' THEN 3
    WHEN condition = 'good' THEN 4
    WHEN condition = 'fair' THEN 5
    WHEN condition = 'salvage' THEN 6
END AS condition,
CASE
    WHEN cylinders = '12 cylinders' THEN 12
    WHEN cylinders = '10 cylinders' THEN 10
    WHEN cylinders = '8 cylinders' THEN 8
    WHEN cylinders = '6 cylinders' THEN 6
    WHEN cylinders = '5 cylinders' THEN 5
    WHEN cylinders = '4 cylinders' THEN 4
    WHEN cylinders = '3 cylinders' THEN 3
    WHEN cylinders = '2 cylinders' THEN 2
    ELSE cylinders
END AS cylinders,
CASE
    WHEN fuel = 'gas' THEN 1
    WHEN fuel = 'electric' THEN 2
    WHEN fuel = 'diesel' THEN 3
    WHEN fuel = 'hybrid' THEN 4
    WHEN fuel = 'other' THEN 5
END AS fuel,
odometer,
CASE
    WHEN transmission = 'manual' THEN 1
    WHEN transmission = 'automatic' THEN 2
    WHEN transmission = 'other' THEN 3
END AS transmission,
CASE
    WHEN drive = 'rwd' THEN 1
    WHEN drive = 'fwd' THEN 2
    WHEN drive = '4wd' THEN 3
END AS drive,
CASE
    WHEN paint_color = 'black' THEN 1
    WHEN paint_color = 'blue' THEN 2
    WHEN paint_color = 'brown' THEN 3
    WHEN paint_color = 'custom' THEN 4
    WHEN paint_color = 'green' THEN 5
    WHEN paint_color = 'grey' THEN 6
    WHEN paint_color = 'orange' THEN 7
    WHEN paint_color = 'purple' THEN 8
    WHEN paint_color = 'red' THEN 9
    WHEN paint_color = 'silver' THEN 10
    WHEN paint_color = 'white' THEN 11
    WHEN paint_color = 'yellow' THEN 12
END AS paint_color,
lat,
ABS(long)

 FROM craigslistVehiclesFull 
 WHERE city = 'losangeles'
 AND manufacturer IS NOT NULL
 AND condition IS NOT NULL
 AND cylinders IS NOT NULL
 AND cylinders != 'other'
 AND odometer IS NOT NULL
 AND transmission IS NOT NULL
 AND drive IS NOT NULL
 AND paint_color IS NOT NULL
 AND lat IS NOT NULL
 AND long IS NOT NULL
 
 AND odometer <= 440000
 AND price >= 500
 AND price <= 219000
 
 
"""
attributes = pd.read_sql(query, conn)
print(attributes)

      price  year  manufacturer  condition  cylinders  fuel  odometer  \
0      5450  2004            31          3          8     1    207000   
1     12800  2016            37          3          4     4     34000   
2      5995  2012            18          3          6     1     50000   
3     24000  2006            15          4          8     3    203000   
4       500  2002            32          4          6     1    175235   
...     ...   ...           ...        ...        ...   ...       ...   
1798  26900  2012            21          3          6     1     79600   
1799   1100  1995            27          5          8     1    133000   
1800   9500  2000             8          3          8     1     80000   
1801   2000  1996            24          3          8     1         0   
1802  17500  2015            34          6          6     3     39000   

      transmission  drive  paint_color        lat   ABS(long)  
0                2      3            9  34.167200  118.3989

(d) I split the data into training/test sets using an 80%/20% split. Note that the train_test_split function shuffles the data by default so no additional parameters are needed.

In [3]:
#80-20 train/test split
train, test = train_test_split(attributes, test_size=0.2)

print(len(train))
print(len(test))

#convert training data into train_X and train_y
train_features = list(train.columns[1:])
train_y = train["price"]
train_x = train[train_features]

#convert testing data into test_X and test_y
test_features = list(test.columns[1:])
test_y = test["price"]
test_x = test[test_features]

1442
361


(e) I trained the decision tree by first optimizing the hyperparameters max_depth and min_samples. This helps improve the accuracy of the tree while also reducing overfitting. 10-fold cross validation is done using 8 threads to complete the evaluation. 

Note that I used the absolute mean earror for scoring. I'm not sure if this has an impact on accuracy but I did it to have an easy to understand output value (the dollar difference between the predicted & actual price).

Then, I output the hyperparameter values and training score of the best tree from the batch.

In [4]:
param_grid = {
    'max_depth':range(1,25),
    'min_samples_split':range(2,25),
    'min_samples_leaf':range(1,25),
}
model = GridSearchCV(tree.DecisionTreeRegressor(), param_grid, cv=10, n_jobs=8, scoring='neg_mean_absolute_error')
model.fit(train_x, train_y)
print('Best Params:')
print(model.best_params_)
print('Best CV Score:')
print(-model.best_score_)

Best Params:
{'max_depth': 10, 'min_samples_leaf': 7, 'min_samples_split': 15}
Best CV Score:
4857.768291763836


c:\users\graham\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


(g) I then stored the best decision tree created by the training process and evaluated it using the testing data set, again using 10-fold cross validation and mean absolute error. The scores output here are close to the best score achieved during training, which shows that the tree isn't overfitting the data.

The best prediction value I could get with this method was about 5000 absolute error, so prediction values within +/- $5000 of the actual sale price. Which isn't great but isn't awful either.

In [5]:
best = model.best_estimator_
scores = cross_val_score(best, test_x, test_y, cv=10, scoring='neg_mean_absolute_error')
scores

array([-5470.71795269, -8218.50610932, -9281.35415803, -5739.23314641,
       -5366.98537358, -5674.37908026, -6303.39558081, -5240.34747829,
       -6089.24035378, -9929.08964608])

(b2) I decided to try finding the best features using some sklearn functionality. First I calculated the correlation to price of the other features in the dataset. From this we can see that the most important by far is odometer reading, followed by manufacturer & paint color. Interestingly, Year does not seem to be important.

In [6]:
array = attributes.values
X = array[:,1:]
Y = array[:,0]

# feature extraction
test = SelectKBest(score_func=chi2, k='all')
fit = test.fit(X, Y)
# summarize scores
np.array2string(fit.scores_, formatter={'float_kind':'{0:.3f}'.format})

'[43.212 3456.683 185.801 306.821 211.919 32989934.371 29.911 168.921\n 1382.842 21.052 73.595]'

I wrote another query which only includes the top 3 features

In [7]:
query2 = """
SELECT 
price,
odometer,
CASE
	WHEN manufacturer = 'acura' THEN 1
	WHEN manufacturer = 'alfa-romeo' THEN 2
    WHEN manufacturer = 'aston' THEN 3
    WHEN manufacturer = 'aston-martin' THEN 3
    WHEN manufacturer = 'audi' THEN 4
    WHEN manufacturer = 'bmw' THEN 5
    WHEN manufacturer = 'buick' THEN 6
    WHEN manufacturer = 'cadillac' THEN 7
    WHEN manufacturer = 'chev' THEN 8
    WHEN manufacturer = 'chevrolet' THEN 8
    WHEN manufacturer = 'chevy' THEN 8
    WHEN manufacturer = 'chrysler' THEN 9
    WHEN manufacturer = 'datsun' THEN 10
    WHEN manufacturer = 'dodge' THEN 11
    WHEN manufacturer = 'ferrari' THEN 12
    WHEN manufacturer = 'fiat' THEN 13
    WHEN manufacturer = 'ford' THEN 14
    WHEN manufacturer = 'gmc' THEN 15
    WHEN manufacturer = 'harley-davidson' THEN 16
    WHEN manufacturer = 'honda' THEN 17
    WHEN manufacturer = 'hyundai' THEN 18
    WHEN manufacturer = 'infiniti' THEN 19
    WHEN manufacturer = 'infinity' THEN 19
    WHEN manufacturer = 'jaguar' THEN 20
    WHEN manufacturer = 'jeep' THEN 21
    WHEN manufacturer = 'kia' THEN 22
    WHEN manufacturer = 'land rover' THEN 23
    WHEN manufacturer = 'rover' THEN 23
    WHEN manufacturer = 'lexus' THEN 24
    WHEN manufacturer = 'lincoln' THEN 25
    WHEN manufacturer = 'mazda' THEN 26
    WHEN manufacturer = 'mercedes' THEN 27
    WHEN manufacturer = 'mercedes-benz' THEN 27
    WHEN manufacturer = 'mercury' THEN 28
    WHEN manufacturer = 'mini' THEN 29
    WHEN manufacturer = 'mitsubishi' THEN 30
    WHEN manufacturer = 'nissan' THEN 31
    WHEN manufacturer = 'pontiac' THEN 32
    WHEN manufacturer = 'porche' THEN 33
    WHEN manufacturer = 'ram' THEN 34
    WHEN manufacturer = 'saturn' THEN 35
    WHEN manufacturer = 'subaru' THEN 36
    WHEN manufacturer = 'toyota' THEN 37
    WHEN manufacturer = 'vw' THEN 38
    WHEN manufacturer = 'volkswagen' THEN 38
    WHEN manufacturer = 'volvo' THEN 39
END AS manufacturer,
CASE
    WHEN paint_color = 'black' THEN 1
    WHEN paint_color = 'blue' THEN 2
    WHEN paint_color = 'brown' THEN 3
    WHEN paint_color = 'custom' THEN 4
    WHEN paint_color = 'green' THEN 5
    WHEN paint_color = 'grey' THEN 6
    WHEN paint_color = 'orange' THEN 7
    WHEN paint_color = 'purple' THEN 8
    WHEN paint_color = 'red' THEN 9
    WHEN paint_color = 'silver' THEN 10
    WHEN paint_color = 'white' THEN 11
    WHEN paint_color = 'yellow' THEN 12
END AS paint_color

 FROM craigslistVehiclesFull 
 WHERE city = 'losangeles'
 AND manufacturer IS NOT NULL
 AND odometer IS NOT NULL
 AND paint_color IS NOT NULL
 
 AND odometer <= 440000
 AND price >= 500
 AND price <= 219000
 
 
"""
attributes2 = pd.read_sql(query2, conn)
print(attributes2)

      price  odometer  manufacturer  paint_color
0      5450    207000            31            9
1     12800     34000            37            9
2      5995     50000            18            6
3     24000    203000            15           11
4       500    175235            32            1
...     ...       ...           ...          ...
2892  17500     39000            34           10
2893  17790     43917            17           10
2894  18490     45617            17            1
2895  14950     22112            14           11
2896  22985     76637            14            2

[2897 rows x 4 columns]


Then, I repeated the training & testing process to create a new decision tree based on the most important features.

In [8]:
#80-20 train/test split
train, test = train_test_split(attributes2, test_size=0.2)

print(len(train))
print(len(test))

#convert training data into train_X and train_y
train_features = list(train.columns[1:])
train_y = train["price"]
train_x = train[train_features]

#convert testing data into test_X and test_y
test_features = list(test.columns[1:])
test_y = test["price"]
test_x = test[test_features]

2317
580


In [9]:
param_grid = {
    'max_depth':range(1,25),
    'min_samples_split':range(2,25),
    'min_samples_leaf':range(1,25),
}
model = GridSearchCV(tree.DecisionTreeRegressor(), param_grid, cv=10, n_jobs=8, scoring='neg_mean_absolute_error')
model.fit(train_x, train_y)
print('Best Params:')
print(model.best_params_)
print('Best CV Score:')
print(-model.best_score_)

Best Params:
{'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best CV Score:
6135.452573501997


However, at the end the performance was about the same (or slightly worse). Interestingly though, the new classifier does seem to perform slightly better when evaluated against the testing data set, which suggests that including too many features increases the risk of overfitting.

In [10]:
best = model.best_estimator_
scores = cross_val_score(best, test_x, test_y, cv=10, scoring='neg_mean_absolute_error')
scores

array([-6308.51122502, -5087.97576032, -6152.07146792, -4991.37823844,
       -5648.91814976, -4851.35657047, -5231.59691329, -5174.85726106,
       -6493.50476738, -8027.68881262])

### Conclusion
This was a fun project for me because it involved working with regression, whereas my previous Datamining project was based on classifiction. It was interesting to see how the techniques differ and where they are the same. 

The final performance of the decision trees wasn't great, but I have a lot of ideas for how performance could be improved. Such as focusing on lower-priced vehicles which seem to have less variance, or doing more work to fine tune the feature selection and hyperparameters. Also, it would be interesting to try out other machine learning techniques on this dataset, such as the Random Forests described in Question 1, or SVM (Support Vector Machine) techniques, which we found had the best performance in my Datamining project.